#Submitted by:

Name: Stephanie Joy R. Susada
Section: S11

**1D Stencil in CUDA**

In [22]:
%%writefile 1d-stencil.cu

#include <cuda_profiler_api.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

//c Kernel
void stencilOperation(int n, int* X, long long int* Y) {
	int i;
	for (i = 3; i < n - 3; i++) {
		Y[i] = X[i - 3] + X[i - 2] + X[i - 1] + X[i] + X[i + 1] + X[i + 2] + X[i + 3];
	}
}
// Kernel function
__global__ void stencilKernel(const int* X, int* Y, int ARRAY_SIZE)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    if (i >= 3 && i < ARRAY_SIZE - 3)
    {
        Y[i] = X[i-3] + X[i-2] + X[i-1] + X[i] + X[i+1] + X[i+2] + X[i+3];
    }
}

int main()
{
    int ARRAY_SIZE = pow(2,26);
    int count = 0, i;
    int* h_X = (int*)malloc(ARRAY_SIZE * sizeof(int));
    int* h_Y = (int*)malloc(ARRAY_SIZE * sizeof(int));
    int* d_X;
    int* d_Y;
    long long int* C_ans = (long long int*)malloc(ARRAY_SIZE * sizeof(long long int));

    //For testing only
    for(int i = 0; i < ARRAY_SIZE; i++){
        h_X[i] = i+1;
    }

    /*
    srand(time(0));
    for(int i = 0; i < ARRAY_SIZE; i++){
        h_X[i] = (rand() % 10) + 1;
    }*/

    cudaMalloc((void**)&d_X, ARRAY_SIZE * sizeof(int));
    cudaMalloc((void**)&d_Y, ARRAY_SIZE * sizeof(int));

    cudaMemcpy(d_X, h_X, ARRAY_SIZE * sizeof(int), cudaMemcpyHostToDevice);

    int blockSize = 256;
    int gridSize = (ARRAY_SIZE + blockSize - 1) / blockSize;

    stencilKernel<<<gridSize, blockSize>>>(d_X, d_Y, ARRAY_SIZE);
    cudaMemcpy(h_Y, d_Y, ARRAY_SIZE * sizeof(int), cudaMemcpyDeviceToHost);

    // Print output
    int counter = 0;

   for (int i = 3; i < ARRAY_SIZE - 3 && counter < 5; i++)
   {
    printf("%d = %d\n", i-2, h_Y[i]);
    counter++;
   }

   stencilOperation(ARRAY_SIZE, h_X, C_ans);

   for(i = 3; i< ARRAY_SIZE -3; i++){
    if(C_ans[i] != h_Y[i]){
      count++;
    }
   }

   if(count != 0){
    printf("\nThere is %d error\n\n", count);
   } else ("\n0 error. Output is correct\n\n");

    cudaFree(d_X);
    cudaFree(d_Y);


    free(h_X);
    free(h_Y);

    return 0;
}

Overwriting 1d-stencil.cu


In [23]:
%%shell
nvcc 1d-stencil.cu -o 1d-stencil

1d-stencil.cu(77): warning #174-D: expression has no effect

1d-stencil.cu(77): warning #174-D: expression has no effect



In [24]:
%%shell
nvprof ./1d-stencil

==7640== NVPROF is profiling process 7640, command: ./1d-stencil
1 = 28
2 = 35
3 = 42
4 = 49
5 = 56
==7640== Profiling application: ./1d-stencil
==7640== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   73.22%  172.04ms         1  172.04ms  172.04ms  172.04ms  [CUDA memcpy DtoH]
                   25.49%  59.891ms         1  59.891ms  59.891ms  59.891ms  [CUDA memcpy HtoD]
                    1.29%  3.0212ms         1  3.0212ms  3.0212ms  3.0212ms  stencilKernel(int const *, int*, int)
      API calls:   51.34%  236.43ms         2  118.21ms  60.076ms  176.35ms  cudaMemcpy
                   48.02%  221.14ms         2  110.57ms  501.48us  220.64ms  cudaMalloc
                    0.59%  2.7263ms         2  1.3631ms  430.41us  2.2959ms  cudaFree
                    0.03%  119.57us       101  1.1830us     137ns  50.270us  cuDeviceGetAttribute
                    0.01%  34.375us         1  34.375us  34.375us  34.375us  cu